In [ ]:
# 1. 라이브러리 가져오기
import pymysql

# 2. 접속하기
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')

# 3. 커서 가져오기
cursor = db.cursor()

sql = """
    create table bookstore1 (
        title varchar(200) not null,
        genre varchar(50),
        ranks varchar(20),
        author varchar(20),
        publs varchar(20),
        publ_date varchar(20),
        e_book varchar(20),
        low_price varchar(20),
        price varchar(20),        
        primary key(title)
    );
"""

# 5. SQL 구문 실행하기
cursor.execute(sql)

# 6. DB에 Complete 하기
db.commit()

# 7. DB 연결 닫기
db.close()

In [ ]:
import requests
import time
from selenium import webdriver

import pymysql
db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
cursor = db.cursor()

options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.115 Safari/537.36')
from bs4 import BeautifulSoup


driver = webdriver.Chrome(options=options)
for page in range(1, 2): #~280
    driver.get('https://search.shopping.naver.com/book/search/category?bookTabType=NEW_BOOK&catId=50005542&pageIndex={}&pageSize=40'.format(page))
    url = 'https://search.shopping.naver.com/book/search/category?bookTabType=NEW_BOOK&catId=50005542&pageIndex={}&pageSize=40'.format(page)
    headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
    res = requests.get(url, headers) 
    res.raise_for_status()
    
    #스크롤    
    prev_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(1)
        curr_height = driver.execute_script("return document.body.scrollHeight")
        
        if curr_height == prev_height:
            break

        prev_height = curr_height    
    print("페이지", page) 

    soup = BeautifulSoup(driver.page_source, "lxml")

    titles = soup.find_all('span',{'class' : 'bookListItem_text__bglOw'}) #제목
#    genres = soup.find_all('div',{'class' : 'bookListItem_feature__txTlp'}) #장르. 순위
    genres_con = soup.find_all('div',{'class' : 'bookListItem_text_area__hF892'}) #버전2 장르
    authors = soup.find_all('span',{'class' : 'bookListItem_author__fdjcn'}) #작가
    publs = soup.select('span.bookListItem_detail_publish__FgPYQ') #출판사
    ebooks = soup.select('div.bookListItem_sub_info__AfkOO') #도서,e북, 가격
    
    count = 0
    for title,genre_con,author, publ, ebook in zip(titles,genres_con, authors, publs, ebooks):
        count +=1
        print(count)
        
        #제목
        title = title.get_text()
        print(title)
        
        #장르, 순위
        genres = genre_con.select('div.bookListItem_feature__txTlp')
        for genre in genres:
            genre = genre.get_text()
            
            if genre :
                j, r =(genre.split()) 
                print('장르'+ ' : ' +j)
                print('순위 '+': ' +r)
            
        #작가
        a,b = author.get_text().split('작가')
        print("작가 :" +b)
        
         #출판
        pub = publ.get_text()
        if pub.find('20') != -1:
            pub_sa= pub[:-11].lstrip("출판") #출판사
            pub_date=pub[-11:] #날짜
            print(pub_sa)
            print(pub_date)
        else:
            pub_sa = pub.lstrip("출판") #출간날짜가 없을때 출판사
            pub_date = "출간날짜 없음"
            print(pub_sa) 
            print(pub_date)        
                     
        # 도서, e북 : 판매처
        eboo= ebook.get_text()
        if eboo.find('도서' and 'e북') != -1:
            book_ebook = "도서 & e북"
            print(book_ebook)
        elif eboo.find('도서') != -1:
            book_ebook = "도서"
            print(book_ebook)
        elif eboo.find('e북') != -1:
            book_ebook = 'e북'
            print(book_ebook)
        else:
            book_ebook = '판매없음'
            print(book_ebook) 
        #판매가격
        ebo= ebook.get_text()
        a= ebo.find('최저')
        b= ebo.find('구매')
        c= ebo.find('신고')
        e= ebo.find('e북')
        if ebo.find('최저'and '구매') != -1:
            book_price = ebo[a+3:e] 
            e_price = ebo[b+3:c]
            print("도서최저 : " + book_price) #최저
            print("e북구매 : " +  e_price) #구매
            print()

        else :
            book_price = ebo[a+3:c].rstrip("원")
            e_price = "ebook판매 안함"
            print("도서최저 : " + book_price) #최저
            print(e_price)
            print()
        sql = """INSERT INTO bookstore1 VALUES(
        "{0}", "{1}", "{2}", "{3}", "{4}", "{5}", "{6}", "{7}", "{8}");""".format(title, j, r, b, pub_sa, pub_date, book_ebook, book_price, e_price)
        cursor.execute(sql)

db.commit()
cursor.close()
db.close()
    
        


In [ ]:
driver.close()